In [1]:
%load_ext rpy2.ipython

In [2]:
import pandas as pd
import numpy as np
import pickle_utils as pu
%R library("ggplot2")
%R install.packages("mice")
%R library("mice")
%R install.packages("lattice")
%R library("lattice")
%R install.packages("VIM")
%R library("VIM")
%R install.packages("missForest")
%R library("missForest")
%R install.packages("doParallel")
%R library("doParallel")
%R install.packages("foreach")
%R library("foreach")

from rpy2.ipython.rmagic import converter

NUMBER_IMPUTATIONS = 5

/home/adria/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Installing package into ‘/home/adria/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/home/adria/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: trying URL 'https://mirror.las.iastate.edu/CRAN/src/contrib/mice_2.30.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/home/adria/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Content type 'application/x-gzip'
  warnings.warn(x, RRuntimeWarning)
/home/adria/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning:  length 648634 bytes (633 KB)

  warnings.warn(x, RRuntimeWarning)
/home/adria/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: =
  warnings.warn(x, RRuntimeWarning)
/home/adria/venv/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

  w

In [3]:
# Register parallel backend
%R registerDoParallel(makeCluster(8))

R object with classes: ('function',) mapped to:
<SignatureTranslatedFunction - Python:0x7ff0cdd6c8c8 / R:0xbec6b38>

In [3]:
table_headers = ["mean_profile", "std_profile", "kurt_profile", "skew_profile", "mean_dm_snr", "std_dm_snr", "kurt_dm_snr", "skew_dm_snr", "label"]
table_dtypes = dict((h, np.float) for h in table_headers)
table_dtypes['label'] = np.bool
original_df = pd.read_csv("HTRU_2.csv", header=None,
                 names=table_headers,
                 dtype=table_dtypes).drop("label", axis=1)
print("number of data points:", original_df.shape[0])

def gondara_rmse_sum(mask_missing, original_df, multiple_imputed_df):
    "RMSE_sum as in Multiple Imputation Using Deep Denoising Autoencoders (Gondara & Wang 2017)"
    assert original_df.shape == multiple_imputed_df[0].shape, "data set shape not matching"
    sq_diff = (original_df-multiple_imputed_df)**2
    sq_diff[:,~mask_missing] = 0
    per_attribute_rmse = np.mean(np.sum(sq_diff, axis=1), axis=0)**.5
    assert per_attribute_rmse.shape == (original_df.shape[1],)
    return np.sum(per_attribute_rmse)

number of data points: 17898


In [4]:
def rmse_sum(mask_missing, original_df, multiple_imputed_df):
    assert original_df.shape == multiple_imputed_df[0].shape, "data set shape not matching"
    sq_diff = (original_df-multiple_imputed_df)**2
    mse = np.sum(sq_diff*mask_missing)/np.sum(mask_missing)
    return mse**.5

# Ampute data set and normalise
That is, introduce missing values into it

In [5]:
def make_random_missing(dataset_):
    "Make dataset miss at random as described in paper"
    dataset = dataset_.copy()
    missing = np.random.rand(len(dataset)) < 0.2
    for i in np.nonzero(missing)[0]:
        r = np.arange(dataset.shape[1])
        np.random.shuffle(r)
        dataset.values[i,r[:(r.shape[0]//2)]] = np.nan
    return dataset

In [6]:
missing_df = make_random_missing(original_df)
# Normalise dataset with only the missing data
normalise_mean = missing_df.mean()
normalise_std = missing_df.std()
missing_df = (missing_df - normalise_mean) / normalise_std
original_df = (original_df - normalise_mean) / normalise_std
assert not np.any(np.isnan(original_df.values)), "Not enough non-NaN values"

# Impute using MICE

In [29]:
%%R -i missing_df -i NUMBER_IMPUTATIONS
# Impute using MICE
imputed_object <- mice(missing_df, m=NUMBER_IMPUTATIONS, maxit=50, meth='pmm', seed=500)


 iter imp variable
  1   1  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  1   2  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  1   3  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  1   4  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  1   5  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  2   1  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  2   2  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  2   3  mean_profile  std_profile  kurt_profile  skew_profile  mean_dm_snr  std_dm_snr  kurt_dm_snr  skew_dm_snr
  2   4  mean_profile  std_profile  kurt_profile  skew_profile  mean

In [30]:
%%R -o r_datasets
# Export MICE's output to Python
r_datasets = lapply(1:NUMBER_IMPUTATIONS, function(v) return(complete(imputed_object, v)))

In [31]:
mice_imputed_data_df = list(converter.ri2py(df) for df in r_datasets)

In [91]:
pu.dump(mice_imputed_data_df, "mice_imputed_data_df.pkl.gz")

In [160]:
rmse_sum(np.isnan(missing_df.values), original_df.values, list(d.values for d in mice_imputed_data_df))

1.1217983156562152

# Impute using MissForest

In [34]:
%%R -o mf_imp
mf_imp <- missForest(missing_df, parallelize='variables')$ximp

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 3 in progress...done!
  missForest iteration 4 in progress...done!
  missForest iteration 5 in progress...done!
  missForest iteration 6 in progress...done!
  missForest iteration 7 in progress...done!


In [161]:
pu.dump([mf_imp], "miss_forest_imputed_data_df.pkl.gz")
#mf_imp = pu.load('miss_forest_imputed_data_df.pkl.gz')[0]
rmse_sum(np.isnan(missing_df.values), original_df.values, [mf_imp.values]*5)

0.44275588795566695

# Neural Network models

In [81]:
pu.dump((np.isnan(missing_df.values), original_df.values), "dataset.pkl.gz")

In [79]:
nn_imputed_data = pu.load("autoencoder_iterate_mcmc.pkl.gz")
_df = pd.DataFrame(dict(zip(original_df.columns, (nn_imputed_data[0][:,i] for i in range(nn_imputed_data[0].shape[1])))))
#_df[list(original_df.columns)]

In [143]:
d = least_squares_dataset(list(d.values for d in mice_imputed_data_df))
nn_imputed_data[0][0,:], d[0,:]

(array([ 1.15763009,  1.33865881, -0.67551243, -0.40109685, -0.31750214,
        -0.37285098, -0.08352529, -0.29369053], dtype=float32),
 array([ 1.15763003,  1.33865881, -0.67551246, -0.40109685, -0.31750215,
        -0.37285098, -0.0835253 , -0.29369053]))

In [163]:
nn_imputed_data = pu.load("vae_iterate_mcmc.pkl.gz")
rmse_sum(np.isnan(missing_df.values), original_df.values, nn_imputed_data)

3.9913386619700288

# GP-LVM

In [ ]:
import GPflow
from GPflow import ekernels
from GPflow import kernels

ignored_df = missing_df.dropna()
n_latent_dims = 5
n_inducing_points = 100
n_training_points = len(ignored_df)
X_mean = GPflow.gplvm.PCA_reduce(ignored_df.values, n_latent_dims)
X_var = 0.1*np.ones((n_training_points, n_latent_dims))
Z = np.random.permutation(X_mean.copy())[:n_inducing_points]

k = ekernels.RBF(n_latent_dims, ARD=True)
m = GPflow.gplvm.BayesianGPLVM(X_mean=X_mean, X_var=X_var, Y=ignored_df.values,
                                kern=k, M=n_inducing_points, Z=Z)
m.likelihood.variance = 0.01
m.optimize(disp=True, maxiter=100)


In [172]:
lvm.posterior